In [ ]:
from ase.io import read
from gpaw import GPAW, FermiDirac, PW
from ase.visualize import view
import numpy as np
import pandas as pd
from dftd4.ase import DFTD4
from dftd3.ase import DFTD3
from ase.build import fcc100

gp = GPAW(mode=PW(500),
          xc='PBE',
          kpts=(3, 3, 1),
          occupations=FermiDirac(0.1),
          txt='gpaw.txt',)


atoms = fcc100('Cu', size=(1,1,1),vacuum=5)

view(atoms)
#atoms.calc = gp
atoms.calc = SumCalculator([gp, DFTD4(method='PBE', damping='d3bj')])
print('D4: ',atoms.get_potential_energy())
atoms.calc= SumCalculator([gp, DFTD3(method='PBE',damping = 'd3bj')])    
print('D3: ',atoms.get_potential_energy())

D4:  -2.4850464555970335
D3:  -2.471274320241147


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Load the DataFrame
df = pd.read_excel('ecut_convergence_results_fixed.xlsx')

# Get unique metal-functional combinations
combinations = df[['Metal', 'Functional']].drop_duplicates().reset_index(drop=True)

# Plot setup
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(12, 18), sharex=True)
axes = axes.flatten()
sns.set(style='whitegrid')

# Plot each metal-functional combination
for i, (metal, functional) in combinations.iterrows():
    subset = df[(df['Metal'] == metal) & (df['Functional'] == functional)]
    ax = axes[i]
    sns.lineplot(data=subset, x='Ecut (eV)', y='Energy (eV)', marker='o', ax=ax)
    ax.set_title(f'{metal} - {functional}')
    ax.set_xlabel('Ecut (eV)', fontweight='bold')
    ax.set_ylabel('Energy (eV)', fontweight='bold')

# Adjust layout
plt.tight_layout()
subplot_path = '/mnt/data/All_Metals_Functionals_Ecut_Energy_Subplots.png'
plt.savefig(subplot_path)
plt.close()

subplot_path
